In [ ]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [85.2 kB]
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ge

In [19]:
import bs4
import csv 
import re
import json
import pprint
import requests

#Test時にはGoogleColabにjsonのアップロード必要

"""
urlItem = "https://so2-api.mutoys.com/master/item.json"
session = requests.Session()
data = session.get(urlItem)
jitem = json.loads(data.text)
urlStore = "https://so2-api.mutoys.com/json/sale/all.json"
data = session.get(urlStore)
jst = json.loads(data.text)
print(jitem)
print(jst)
"""
# 自分の店の名前
nameMyShop="いい天気"

# json open load
jopenitem = open('soldout2item.json', 'r', encoding='Shift_JIS')
jitem = json.load(jopenitem)
jopentown = open('soldout2town.json', 'r', encoding='UTF-8')
jtown = json.load(jopentown)
jopenshop = open('soldout2shop.json', 'r', encoding='UTF-8')
jshop = json.load(jopenshop)
jopenst = open('soldout2store.json', 'r', encoding='UTF-8')
jst = json.load(jopenst)
#print(repr(jopenitem))
#print(repr(jopentown))
#print(repr(jopenst))

urlItem = "https://so2-api.mutoys.com/master/item.json"
session = requests.Session()
data = session.get(urlItem)
jitem = json.loads(data.text)
urlStore = "https://so2-api.mutoys.com/json/sale/all.json"
data = session.get(urlStore)
jst = json.loads(data.text)
urlShop = "https://so2-api.mutoys.com/json/shop/all.json"
data = session.get(urlShop)
jshop = json.loads(data.text)



def getItemid(name):
  jitemvals = jitem.values()
  for vitem in jitemvals:
    if vitem["name"] == name:
      ret=vitem["item_id"]
      exit
  return ret

def getItemname(id):
  jitemvals = jitem.values()
  for vitem in jitemvals:
    if vitem["item_id"] == id:
      ret=vitem["name"]
      exit
  return ret

def getShopName(name):
  for shop in jshop:
    if shop["shop_name"] == name:
      return shop

def getShopsItem(itemname):
  liShope=[]
  for vst in jst:
    if vst["item_id"] == getItemid(itemname):
      liShope.append(vst)
  return liShope

def getPos(shop):
  #shopエリア特定  
  jtowns = jtown.values()
  for town in jtowns:
    if town['area_id'] == shop['area_id']:
      shopTown=town
      exit
  posXshop=shop['pos_x']+shopTown['pos_x']
  posYshop=shop['pos_y']+shopTown['pos_y']
  return [posXshop,posYshop]

def getTransTime(shop):
  #(輸送時間[min]) ＝(0.15×(マンハッタン距離)^2+50)/(輸送速度[%])
  myShop=getShopName(nameMyShop)

  posMe=getPos(myShop)
  posShop=getPos(shop)
  trtime=( 0.15 * (abs(posShop[0]-posMe[0])+abs(posShop[1]-posMe[1]) ) **2 +50 ) / 100
  ret = trtime
  return ret

# アイテムについて、輸送時間が一定以下の店を取得
# html(寄付ページ) 解析
soup = bs4.BeautifulSoup(open('soldout2_donrec.html'), 'html.parser')
itemsets=(soup.find_all(style =re.compile("width: 12rem; font-size: small; display: inline-block; text-align: left;")))
between=15
for num in range (0,76,between):
  print("{0}分以上{1}分未満".format(num,num+between))
  for itemset in itemsets:
    #itemset==寄付のセット
    #print(itemset.text)
    buyitemsets=[]
    items=itemset.find_all(class_ =re.compile("icon icon-item icon-item-"))
    if not itemset.text.startswith("[0]"):
      continue
    itemYes=False
    for item in items:
      #短時間のitemが一個も無かったら次のitem
      #itemが全部短時間なら出力
      itemShops=getShopsItem(item["title"])
      shopYes=False
      for shop in itemShops:
        #短時間のshopが一個も無かったら次のitem
        liItemSale=[]
        transTime=getTransTime(shop)
        if num <= getTransTime(shop) < num+between:
          liItemSale.append(getItemname(shop["item_id"]))
          liItemSale.append("{:,}".format((shop["price"])))
          liItemSale.append("{0} 分".format(transTime))
          buyitemsets.append(liItemSale)
          shopYes=True
      if shopYes==False:
        break
    if shopYes==True:
      print(itemset.text)
      for item in items:
        print(item["title"])
      for b in buyitemsets:
        print(b)

csvlist = [] # 配列を作成
for link in links: # aタグのテキストデータを配列に格納
    sample_txt = link.text
    csvlist.append(sample_txt)

# CSVファイルを開く。ファイルがない場合は新規作成
f = open("output_sample.csv", "w")
writecsv = csv.writer(f, lineterminator='\n')

writecsv.writerow(csvlist) # 出力

f.close() # CSVファイルを閉じる

0分以上15分未満
[0]一品物セット
とっておきのハンマー
フィン・マックールの剣
['とっておきのハンマー', '500,000', '0.5375 分']
['とっておきのハンマー', '100,000', '0.716 分']
['フィン・マックールの剣', '550,000', '3.9559999999999995 分']
[0]拾い物セット
短剣
血の剣
サイクロプスの棍棒
エルフの弓
['短剣', '55,000', '11.853499999999999 分']
['短剣', '9,000', '0.884 分']
['血の剣', '60,000', '0.6815000000000001 分']
['サイクロプスの棍棒', '600,000', '12.921499999999998 分']
['エルフの弓', '50,000', '0.5375 分']
['エルフの弓', '50,000', '0.5375 分']
[0]ギョギョギョセット
魚
うおのぼり
['魚', '400', '11.853499999999999 分']
['魚', '250', '13.4735 分']
['魚', '250', '14.905999999999999 分']
['魚', '220', '14.0375 分']
['うおのぼり', '1,500,000', '12.921499999999998 分']
[0]釣れちゃったセット
ヌシ
サーペント
['ヌシ', '10,000,000', '0.6214999999999999 分']
['サーペント', '300,000', '9.164 分']
[0]甘い汁セット
ハチミツ
メィプルシロップ
['ハチミツ', '5,500', '4.7135 分']
['メィプルシロップ', '3,500', '4.7135 分']
[0]祝杯セット
プォンジュース
いちご牛乳
コーヒー牛乳
['プォンジュース', '1,000', '9.3935 分']
['いちご牛乳', '10,000', '9.164 分']
['コーヒー牛乳', '3,500', '0.7535 分']
[0]収集家セット
砂
鳥の羽根
石
色石
['砂', '200', '3.2735 分']
['砂', '100', '0.68150

NameError: ignored